In [1]:
%load_ext autoreload
%autoreload 2

import pymc3 as pm
import pandas as pd
import numpy as np
import arviz as az
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from matplotlib import pyplot as plt
from covid.models.generative import GenerativeModel
from covid.data import summarize_inference_data
%config InlineBackend.figure_format = 'retina'

from covid.data import get_and_process_covidtracking_data_il, get_and_process_covidtracking_data_ilsev, summarize_inference_data

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-iw7i_4sj because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
dfcit = get_and_process_covidtracking_data_il(run_date=pd.Timestamp.today()-pd.Timedelta(days=1), norm=False, cities=True)
cmap = pd.read_csv("Cities_stats_2019.csv", dtype = str)
cmap.Region.fillna(cmap.Subdistrict, inplace=True)
cmap = cmap.groupby("Region")["City_code"].apply(list).reset_index().set_index("Region")
cmap.head(10)
trans = pd.read_csv("region_names.csv", dtype = str)
trans.set_index("Region", inplace=True)
trans.head(10)
trans.loc["111"]["Region_name"] 

'הרי יהודה'

In [ ]:
# Run the Rt computation for each region
for region in list(cmap.index.values):
    cities = set(cmap.loc[region]['City_code']) & set(list(dfcit.index.unique(0)))
    model = dfcit.loc[cities].groupby('date').sum()
    if sum(model['positive']) < 1000:
        continue
    gm = GenerativeModel(cities, model, buffer_days = 10, delay=2, sigma=0.07)
    gm.sample()
    result = summarize_inference_data(gm.inference_data)

    fig, ax = plt.subplots(figsize=(10,5))
    ax.set_title(region + " " + trans.loc[region]["Region_name"][::-1])
    result.test_adjusted_positive.plot(c="g", label="Test-adjusted")
#resultbb.test_adjusted_positive_raw.plot(c="g", alpha=.5, label="Test-adjusted (raw)", style="--")
    result.infections.plot(c="b", label="Infections")
    gm.observed.positive.plot(c='r', alpha=.7, label="Reported Positives")
#d = gmbb.observed.deaths * 100
#d.plot(c='k', alpha=0.5, label="Reported deaths (x100)")
    fig.set_facecolor('w')
    plt.yscale("log")
    ax.legend();
    fig.savefig("Israel_unnorm_data_" + region + ".svg")
   
    fig, ax = plt.subplots(figsize=(10,5))

    ax.set_title(region + " " + trans.loc[region]["Region_name"][::-1] + f" $R_t$")
    samples = gm.trace['r_t']
    x=result.index
    colmap = plt.get_cmap("Reds")
    ax.fill_between(x, result["upper_80"], result["lower_80"], color=colmap(50), alpha=.8)
    result["median"].plot(c=colmap(90), ls='-', label="Rt using infected in " + region)
    ax.axhline(1.0, c="k", lw=1, linestyle="--")
    ax.set_xlim(right=pd.Timestamp.today()-pd.Timedelta(days=10))
    fig.set_facecolor('w')
    plt.grid(which='major', axis='y', ls=':')
    fig.savefig("Israel_unnorm_Rt_est_" + region + ".svg")
    result.to_csv("results_unnormalized_out_" + region + ".csv")

Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, seed, log_r_t]


Sampling 4 chains for 700 tune and 200 draw iterations (2_800 + 800 draws total) took 2157 seconds.


Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, seed, log_r_t]


Sampling 4 chains for 700 tune and 200 draw iterations (2_800 + 800 draws total) took 2011 seconds.


Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, seed, log_r_t]


...

Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, seed, log_r_t]


Sampling 4 chains for 700 tune and 200 draw iterations (2_800 + 800 draws total) took 1955 seconds.
The estimated number of effective samples is smaller than 200 for some parameters.


Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, seed, log_r_t]


Sampling 4 chains for 700 tune and 200 draw iterations (2_800 + 800 draws total) took 1836 seconds.


Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, seed, log_r_t]
